In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

선형 회귀 분석은 가장 기본적인 머신러닝 방법이며, 여러 분야에서 사용된다.

선형 회귀 분석은 피쳐 데이터와 타겟 데이터 간의 선형 관계를 파악하는 알고리즘으로 피쳐 데이터 x와 타겟 데이터 y 사이에 선형 관계가 존재할 때 이를 수식화 하면 다음과 같다.

$$y = wx + b$$

데이터 x가 주어질 때, 데이터 x에 가중치 w를 곱하고 y절편 b를 더하면 타겟 데이터를 얻을 수 있다. 선현 회귀에서 해야 할 일은 피쳐 데이터 x와 타겟 데이터 y를 이용해서 가중치 w와 y절편 b를 구하는 것이다.

위에서는 피쳐 개수가 1개라 가정했는데, 이를 일반화시켜 $p$개의 피쳐를 가진 데이터라고 가정하면 데이터셋을 구성하는 각 데이터 포인트는 $p$개의 피쳐로 구성되므로 $x = (x_1,x_2,x_3, \dots, x_p)$라고 표현할 수 있다. $x_i$는 $i$번째 데이터 행을 열벡터 형태로 표현한 것이다. 그리고 선형 회귀 모형은 아래와 같다.

$$f(x_i) = \hat y = w_1x_1 + w_2x_2 + \dots + w_px_p + b$$

이때, $w = (w_1, w_2, \dots, w_p)^T$를 가중치(weight)라고 부른다. 각 가중치 요소 하나하나가 우리가 구하려는 파라미터(parameter)이며, 파라미터 값은 예측값에 영향을 미친다. 즉, 파라미터 값에 따라 예측이 달라진다.

가중치는 트레이닝 데이터로부터 최소 제곱법(least squares estimator)을 사용해 구할 수 있다. 최소 제곱법은 오차의 제곱합이 최소가 되는 추정량을 구하는 방법이다.

보스턴 집값 데이터를 예측하는 모형을 만들어 본다.  
또한 선형 회귀 분석뿐만 아니라 릿지 회귀 분석, 라쏘 회귀 분석, 엘라스틱 넷을 사용한 결과를 비교해 본다.

In [2]:
# 데이터 불러오기
from sklearn import datasets # 보스턴 집값 데이터를 사용하기 위해 import 한다.
raw_boston = datasets.load_boston() # 보스턴 집값 데이터를 불러온다.

In [3]:
# 피쳐/타겟 데이터 지정
X = raw_boston.data # 보스턴 집값 피쳐 데이터를 저장한다.
y = raw_boston.target # 보스턴 집값 타겟 데이터를 저장한다.

In [4]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝/테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=1) # 트레이닝 데이터와 테스트 데이터로 분할한다.
print(X_tn.shape, X_te.shape)

(379, 13) (127, 13)


In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn) # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [6]:
# 데이터 학습(선형 회귀 분석)
from sklearn.linear_model import LinearRegression # 선형 회귀 분석 알고리즘을 사용하기 위해 import 한다.
clf_lr = LinearRegression() # 선형 회귀 분석 모델 객체를 만든다.
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 선형 회귀 분석 알고리즘을 학습시킨다.
clf_lr.fit(X_tn_std, y_tn)

LinearRegression()

In [7]:
# 선형 회귀 분석 계수(가중치), 상수항(바이어스) 확인
print(clf_lr.coef_) # 가중치
print(clf_lr.intercept_) # 바이어스

[-1.07145146  1.34036243  0.26298069  0.66554537 -2.49842551  1.97524314
  0.19516605 -3.14274974  2.66736136 -1.80685572 -2.13034748  0.56172933
 -4.03223518]
22.344591029023768


In [8]:
# 데이터 학습(L2 제약식 적용, 릿지 회귀 분석)
from sklearn.linear_model import Ridge # 릿지 회귀 분석 알고리즘을 사용하기 위해 import 한다.
# 릿지 회귀 분석 모델을 만든다.
# alpha 옵션의 기본값은 1이고 반드시 양수이어야 하며, 값이 클수록 강한 제약식을 의미한다.
clf_ridge = Ridge(alpha=1)
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 릿지 회귀 분석 알고리즘을 학습시킨다.
clf_ridge.fit(X_tn_std, y_tn)

Ridge(alpha=1)

In [9]:
# 릿지 회귀 분석 계수, 상수항 확인
print(clf_ridge.coef_) # 가중치
print(clf_ridge.intercept_) # 바이어스

[-1.05933451  1.31050717  0.23022789  0.66955241 -2.45607567  1.99086611
  0.18119169 -3.09919804  2.56480813 -1.71116799 -2.12002592  0.56264409
 -4.00942448]
22.344591029023768


In [14]:
# 데이터 학습(L1 제약식 적용, 라쏘 회귀 분석)
from sklearn.linear_model import Lasso # 라쏘 회귀 분석 알고리즘을 사용하기 위해 import 한다.
# 라쏘 회귀 분석 모델을 만든다.
# 릿지 회귀 분석과 마찬가지로 alpha 값은 제약의 정도를 나타내며 1을 기본값으로 가진다.
clf_lasso = Lasso(alpha=0.01)
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 라쏘 회귀 분석 알고리즘을 학습시킨다.
clf_lasso.fit(X_tn_std, y_tn)

Lasso(alpha=0.01)

In [15]:
# 라쏘 회귀 분석 계수, 상수항 확인
print(clf_lasso.coef_) # 가중치
print(clf_lasso.intercept_) # 바이어스

[-1.04326518  1.27752711  0.1674367   0.66758228 -2.41559964  1.99244179
  0.14733958 -3.09473711  2.46431135 -1.60552274 -2.11046422  0.55200229
 -4.00809905]
22.344591029023768


In [17]:
# 데이터 학습(엘라스틱 넷)
from sklearn.linear_model import ElasticNet # 엘라스틱 회귀 분석 알고리즘을 사용하기 위해 import 한다.
# 엘라스틱 회귀 분석 모델을 만든다.
# 옵션값으로 쓰이는 alpha 값은 L1 제약식의 크기와 L2 제약식의 크기의 합을 의미한다.
# l1_ratio 옵션 값은 전체 제약 크기인 alpha 값에서 L1 제약이 차지하는 비율을 의미한다.
# l1_ratio는 비율이므로 0과 1의 사이값을 가지며, l1_ratio=0이라면 L1 제약이 적용되지 않고 L2 제약만 적용되는
# 릿지 회귀 분석을 의미하고 반대로 l1_ratio=1이라면 L1 제약만 적용되고 L2 제약은 적용되지 않으므로 라쏘 회귀
# 분석이 됨을 의미한다.
clf_elastic = ElasticNet(alpha=0.01, l1_ratio=0.01)
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 엘라스틱 회귀 분석 알고리즘을 학습시킨다.
clf_elastic.fit(X_tn_std, y_tn)

ElasticNet(alpha=0.01, l1_ratio=0.01)

In [18]:
# 엘라스틱 회귀 분석 계수, 상수항 확인
print(clf_elastic.coef_) # 가중치
print(clf_elastic.intercept_) # 바이어스

[-1.02916603  1.23681955  0.15236504  0.67859622 -2.34646781  2.02965524
  0.14575132 -2.98592423  2.32013379 -1.48829485 -2.09271972  0.56506801
 -3.9495281 ]
22.344591029023768


In [22]:
# 선형 회귀, 릿지 회귀, 라쏘 회귀, 엘라스틱 회귀로 예측해본다.
pred_lr = clf_lr.predict(X_te_std)
pred_ridge = clf_ridge.predict(X_te_std)
pred_lasso = clf_lasso.predict(X_te_std)
pred_elastic = clf_elastic.predict(X_te_std)
# print(pred_lr.shape, pred_ridge.shape, pred_lasso.shape, pred_elastic.shape)
for i in range(len(pred_lr))[:5]:
    print('선형: {:.5f}, 릿지: {:.5f}, 라쏘: {:.5f}, 엘라스틱: {:.5f}'.format(
        pred_lr[i], pred_ridge[i], pred_lasso[i], pred_elastic[i]
    ))

선형: 32.37817, 릿지: 32.26711, 라쏘: 32.19249, 엘라스틱: 31.99335
선형: 27.95684, 릿지: 27.97205, 라쏘: 27.98937, 엘라스틱: 28.00404
선형: 18.07214, 릿지: 18.08339, 라쏘: 18.04237, 엘라스틱: 18.11898
선형: 21.63167, 릿지: 21.68419, 라쏘: 21.75823, 엘라스틱: 21.81160
선형: 18.93030, 릿지: 18.89663, 라쏘: 18.88655, 엘라스틱: 18.81388


In [26]:
# 모형 평가 - R 제곱값(r2 score)
from sklearn.metrics import r2_score # r2 score를 계산하기 위해 import 한다.
# 실제 타겟 데이터와 모형별 예측값으로 r2 score 값을 게산한다.
# r2 score 값은 0과 1 사이값을 가지며 높을수록 좋은 성능을 의미한다.
print('선형 r2 score: ', r2_score(y_te, pred_lr))
print('릿지 r2 score: ', r2_score(y_te, pred_ridge))
print('라쏘 r2 score: ', r2_score(y_te, pred_lasso))
print('엘라 r2 score: ', r2_score(y_te, pred_elastic))

선형 r2 score:  0.7789410172622855
릿지 r2 score:  0.7789704562726605
라쏘 r2 score:  0.7787621490259894
엘라 r2 score:  0.7787876079239252


In [27]:
# 모형 평가 - MSE
from sklearn.metrics import mean_squared_error # MSE를 계산하기 위해 import 한다.
print('선형 MSE: ', mean_squared_error(y_te, pred_lr))
print('릿지 MSE: ', mean_squared_error(y_te, pred_ridge))
print('라쏘 MSE: ', mean_squared_error(y_te, pred_lasso))
print('엘라 MSE: ', mean_squared_error(y_te, pred_elastic))

선형 MSE:  21.897765396049525
릿지 MSE:  21.894849212618745
라쏘 MSE:  21.915483810504835
엘라 MSE:  21.912961890936874
